# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io/library-webscraping/](https://data-lessons.github.io/library-webscraping/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [1]:
import requests
# 1. Download the webpage
response = requests.get('https://www.sfu.ca/computing/people/faculty.html')
# 2. Save it as a text file (named faculty.txt)
file = open('faculty.txt','w')
file.write(response.text)
file.close()

### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [2]:
import lxml.html 
import pandas as pd

# 1. Open faculty.txt
file = open('faculty.txt','r')
# 2. Parse the HTML page as a tree structure
tree = lxml.html.fromstring(file.read())
# 3. Extract related content from the tree using XPath
info = tree.xpath('//div[contains(@class, "text")]/p[1]')
names = tree.xpath('//div[contains(@class, "text")]/h4[1]')
home= tree.xpath('//div[contains(@class, "text")]/h4[1]')
facultyName=[]
position=[]
for name in names:
    name_rank=name.text_content()
    attr=name_rank.split(',')
    facultyName.append(attr[0].strip())
    position.append(attr[1].split('\n')[0].strip())   
facultyArea=[]
for inf in info:
    area=inf.text_content()
    facultyArea.append(area.split(':')[1].split('\n')[0].strip()) 
start_url = 'http://www.sfu.ca/'
profile = tree.xpath('//div[contains(@class, "text")]/p/a[1]')
start_url = 'http://www.sfu.ca/'
profiles=[]
homepages=[]
for link in profile:
    if(link.text=="Profile & Contact Information"):
        if((str(link.attrib['href']).startswith('/computing')==1)):
            faculty_url = requests.compat.urljoin(start_url, link.attrib['href'])
            profiles.append(faculty_url)
        else:
            profiles.append(link.attrib['href']) 
        if(link.getnext()==None):
            homepages.append('')
        else:
            if(link.getnext().tag=='a'):
                homepages.append(link.getnext().attrib['href'])
            else:
                homepages.append('')
# 4. Save the extracted content as an csv file (named faculty_table.csv)
df = pd.DataFrame()
df['name']=facultyName
df['rank']=position
df['area']=facultyArea
df['profile']=profiles
df['homepage']=homepages
df.to_csv('faculty_table.csv',index=False)

## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 56 faculties, so you need to download 56 web pages in total. 

In [3]:
import requests
import pandas as pd

# 1. Download the profile pages of 56 faculties
# 2. Save each page as a text file
df=pd.read_csv('faculty_table.csv')
pages=[]
for i in df['profile']:            
    response = requests.get(i)
    file = open(i.split('/')[-1].split('.')[0]+'.txt','w')
    pages.append(i.split('/')[-1].split('.')[0]+'.txt')
    ## Saving as Text Page
    file.write(response.text)
    file.close()

### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [15]:
import lxml.html 

# 1. Open each text file and parse it as a tree structure
# 2. Extract the research interests from each tree using XPath
# 3. Add the extracted content to faculty_table.csv
# 4. Generate a new CSV file, named faculty_more_table.csv
interest=[]
for page in pages:
    file = open(page,'r') 
    tree = lxml.html.fromstring(file.read())
    research = tree.xpath('//div/h2[starts-with(.,"Research")]')
    word='['
    var=[]
    if(len(research)>1):      
        for i in research:
            if(i.getnext().tag=='ul'):
                ch=i.getnext().getchildren()
                for j in ch:
                    if(j==ch[-1]):
                        word+=j.text_content().strip()
                    else:
                        word+=j.text_content().strip()+','+' '
            word+=','+' '
        word=word.strip(", ")
        word+=']'
        interest.append(word.replace("\xa0",""))
    elif(len(research)==0):
        interest.append(var)
    else:
        for i in research:
            if(i.getnext().tag=='ul'):
                ch=i.getnext().getchildren()
                for j in ch:
                    if(j==ch[-1]):
                        word+=j.text_content().strip()+']'
                    else:
                        word+=j.text_content().strip()+','+' '
                interest.append(word.replace("\xa0",""))
            else:
                interest.append(var)
df['research_interests']=interest
df.to_csv('faculty_more_table.csv',index=False)

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity [Assignment 1](https://courses.cs.sfu.ca/2018sp-cmpt-733-g1/+a1/).